# Loop 5 LB Feedback Analysis

**LB Score: 70.6761** (same as CV - perfect calibration)

## Key Finding from Snapshot Analysis

Found a snapshot (21145968755) with LB score **70.659959** - better than our current 70.676102!

This is a 0.016 improvement. Let me investigate what data source they used.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def calculate_score_for_csv(csv_path):
    df = pd.read_csv(csv_path)
    total_score = 0
    
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = df[df['id'].str.startswith(prefix)]
        if len(rows) == 0:
            continue
        
        trees = []
        for _, row in rows.iterrows():
            x = Decimal(parse_value(row['x']))
            y = Decimal(parse_value(row['y']))
            angle = Decimal(parse_value(row['deg']))
            
            vertices = []
            for tx, ty in zip(TX, TY):
                vertices.append((float(Decimal(str(tx)) * scale_factor), 
                               float(Decimal(str(ty)) * scale_factor)))
            initial_polygon = Polygon(vertices)
            rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
            translated = affinity.translate(
                rotated,
                xoff=float(x * scale_factor),
                yoff=float(y * scale_factor)
            )
            trees.append(translated)
        
        all_polygons = unary_union(trees)
        bounds = all_polygons.bounds
        side = float(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / float(scale_factor)
        score_n = side**2 / n
        total_score += score_n
    
    return total_score

print("Functions defined")

Functions defined


In [2]:
# Compare our current baseline with the saspav_latest
print("Comparing CSV sources...")
print()

# Our current baseline
current_baseline = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv'

# The better snapshot's source
better_source = '/home/nonroot/snapshots/santa-2025/21145968755/code/external_data/saspav_latest/santa-2025.csv'

print(f"Current baseline: {current_baseline}")
print(f"Better source: {better_source}")
print()

# Quick check - compare first few lines
import subprocess
print("=== Current baseline (first 10 lines) ===")
result = subprocess.run(['head', '-10', current_baseline], capture_output=True, text=True)
print(result.stdout)

print("=== Better source (first 10 lines) ===")
result = subprocess.run(['head', '-10', better_source], capture_output=True, text=True)
print(result.stdout)

Comparing CSV sources...

Current baseline: /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv
Better source: /home/nonroot/snapshots/santa-2025/21145968755/code/external_data/saspav_latest/santa-2025.csv

=== Current baseline (first 10 lines) ===
id,x,y,deg
001_0,s-48.196086194214246,s58.770984615214225,s45.0
002_0,s0.154097069621355887,s-0.038540742694794648,s203.629377730656841550
002_1,s-0.154097069621372845,s-0.561459257305224058,s23.629377730656791812
003_0,s1.123655816140301,s0.781101815992563,s111.125132292893
003_1,s1.23405569584216,s1.275999500663759,s66.370622269343
003_2,s0.641714640229075,s1.180458566613381,s155.13405193710082
004_0,s-0.324747789589372171,s0.132109978088185392,s156.370622145636389178
004_1,s0.315354346242637695,s0.132109978063475492,s156.370622269264089255
004_2,s0.324747789592379210,s-0.732109978069475531,s336.370622269264003990

=== Better source (first 10 lines) ===
id,x,y,deg
001_0,s-48.196086194214246,s58.77

In [3]:
# Calculate scores for both
print("Calculating scores (this may take a minute)...")
print()

current_score = calculate_score_for_csv(current_baseline)
print(f"Current baseline score: {current_score:.6f}")

better_score = calculate_score_for_csv(better_source)
print(f"Better source score: {better_score:.6f}")

print(f"\nImprovement: {current_score - better_score:.6f}")

Calculating scores (this may take a minute)...



Current baseline score: 70.676102


Better source score: 70.659959

Improvement: 0.016143


In [ ]:
# Find which N values are different
print("\nFinding which N values differ...")

current_df = pd.read_csv(current_baseline)
better_df = pd.read_csv(better_source)

differences = []
for n in range(1, 201):
    prefix = f"{n:03d}_"
    
    current_rows = current_df[current_df['id'].str.startswith(prefix)].sort_values('id')
    better_rows = better_df[better_df['id'].str.startswith(prefix)].sort_values('id')
    
    # Compare x values
    current_x = [parse_value(x) for x in current_rows['x'].values]
    better_x = [parse_value(x) for x in better_rows['x'].values]
    
    if current_x != better_x:
        differences.append(n)

print(f"N values that differ: {differences}")
print(f"Total different: {len(differences)}")

In [ ]:
# Calculate per-N scores for both and find improvements
print("\nPer-N score comparison for different N values:")
print("="*60)

def calculate_score_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    if len(rows) == 0:
        return None
    
    trees = []
    for _, row in rows.iterrows():
        x = Decimal(parse_value(row['x']))
        y = Decimal(parse_value(row['y']))
        angle = Decimal(parse_value(row['deg']))
        
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
        translated = affinity.translate(
            rotated,
            xoff=float(x * scale_factor),
            yoff=float(y * scale_factor)
        )
        trees.append(translated)
    
    all_polygons = unary_union(trees)
    bounds = all_polygons.bounds
    side = float(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / float(scale_factor)
    return side**2 / n

for n in differences[:20]:  # Show first 20
    current_n = calculate_score_for_n(current_df, n)
    better_n = calculate_score_for_n(better_df, n)
    improvement = current_n - better_n if current_n and better_n else 0
    print(f"N={n:3d}: current={current_n:.9f}, better={better_n:.9f}, improvement={improvement:.9f}")

In [ ]:
# Copy the better source to our submission
import shutil
shutil.copy(better_source, '/home/submission/submission.csv')
print(f"Copied better source to submission")
print(f"Expected score: {better_score:.6f}")